In [1]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import trends_helpers
import os
import pycountry

In [2]:
os.getcwd()

"g:\\Mi unidad\\BSE - DSDM\\Master's project\\refugees\\notebooks\\trends"

In [3]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [4]:
currencies.head()

,Country Code,Country Name,WHO Region,World Bank Income Group (2018),Currency Name,Currency Code
0,AFG,Afghanistan,EMR,Low,Afghanistan Afghani,AFN
1,ALB,Albania,EUR,Upper‐middle,Albanian Lek,ALL
2,DZA,Algeria,AFR,Upper‐middle,Algerian Dinar,DZD
3,AND,Andorra,EUR,High,Euro,EUR
4,AGO,Angola,AFR,Lower‐middle,Angolan Kwanza,AON


In [5]:
currencies.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country Code                    194 non-null    object
 1   Country Name                    194 non-null    object
 2   WHO Region                      194 non-null    object
 3   World Bank Income Group (2018)  194 non-null    object
 4   Currency Name                   194 non-null    object
 5   Currency Code                   194 non-null    object
dtypes: object(6)
memory usage: 9.2+ KB


In [6]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

# First search as it comes, then we can translate for each origin lang

141

In [7]:
keywords_list = [
 'Congolese Franc',
 'Cape Verde Escudo',
 'Zloty',
 'Belize Dollar',
 'New Zealand Dollar',
 'Ringgit',
 'Rouble',
 'Omani Rial',
 'Ngultrum',
 'Taka',
 'Euro',
 'Norway Krone',
 'Ghana Cedi',
 'Kina',
 'Rufiyaa',
 'Pound Sterling',
 'Gambian Dalasi',
 'Belarusian Ruble',
 'Riel',
 'Leone',
 'Philippines Peso',
 'Brunei Dollar',
 'Namibian Dollar',
 "Pa'anga",
 'Botswana Pula',
 'Moldova Leu',
 'Bulgarian Lev',
 'Baharaini Dinar',
 'Vatu',
 'Malawian Kwacha',
 'Guatemalan Quetzal',
 'Lebanese Pound',
 'Mauritius Rupee',
 'Nakfa',
 'Chilean Peso',
 'North Korean Won',
 'Turkish Lira',
 'Trinidad and Tobago Dollar',
 'New Sheqel',
 'Ethiopian Birr',
 'Egyptian Pound',
 'Syrian Pound',
 'Hungarian Forint',
 'Qatar Rial',
 'Tenge',
 'Rwandan Franc',
 'Indonesian Rupiah',
 'New Ouguiya',
 'Danish Krone',
 'Denar',
 'Balboa',
 'Yemeni Rial',
 'Won',
 'New Dobra',
 'Afghanistan Afghani',
 'Guinea Franc',
 'Burundi Franc',
 'Colombian Peso',
 'Malagasy Ariary',
 'Japanese Yen',
 'Jamaican Dollar',
 'Eastern Caribbean Dollar',
 'Lilangeni',
 'Serbian Dinar',
 'Singapore Dollar',
 'Kenyan Shilling',
 'Iraqi Dinar',
 'Pakistani Rupee',
 'Djibouti Franc',
 'Tala',
 'Lesotho Loti',
 'Bahamian Dollar',
 'Sri Lanka Rupee',
 'Somalian Shilling',
 'Ugandan shilling',
 'Saudi Riyal',
 'Solomon Dollar',
 'Somoni',
 'Hryvnia',
 'Costa Rican Colone',
 'Tugrik',
 'Yuan Renminbi',
 'Iranian Rial',
 'Tunisian Dinar',
 'Liberian Dollar',
 'Swedish Krona',
 'Dram',
 'Seychelles Rupee',
 'Mexican Peso',
 'Angolan Kwanza',
 'Surinamese Dollar',
 'Canadian Dollar',
 'Uruguayan Peso',
 'South Sudanese Pound',
 'Honduran Lempira',
 'Convertible Mark',
 'Guarani',
 'Kip',
 'Comoros Franc',
 'Dominican Peso',
 'Som',
 'Kuna',
 'U.A.Emirates Dirham']

In [8]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [14]:
proxy_list = ['190.61.88.147:8080', 'https://178.33.3.163:8080']



In [17]:
from pytrends.request import TrendReq
import time


# Set up the Google Trends API object
# pytrends = TrendReq()
# I'll define the object within the loop

# Initialize an empty DataFrame to store the results
results = pd.DataFrame()


# Loop through regions and keywords
for keyword in keywords_list:
    for i, country in enumerate(countries):    
        # pytrends = TrendReq(timeout=(10,25), proxies=proxy_list, retries=2, backoff_factor=0.1,    )

        pytrends = TrendReq(tz=360, timeout=(10, 25), retries=2, backoff_factor=0.5, 
                            requests_args={'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'}})
        
# The timeout parameter sets the maximum number of seconds to wait for a response from the API 
# before raising a timeout error.
#The backoff_factor parameter is used for exponential backoff when retrying requests. 
# It specifies the factor by which the waiting time between retries increases on each retry. 
# For example, if backoff_factor is set to 0.4, the waiting time between retries will increase by 40% on
# each retry.
# The retries parameter sets the maximum number of times to retry a request if it fails with a 
# transient error (such as a connection error or server error). If the maximum number of retries 
# is exceeded, an exception will be raised.   
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(4)
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])



Searching: Congolese Franc in AF
Searching: Congolese Franc in AL
Searching: Congolese Franc in DZ
Searching: Congolese Franc in AD
Searching: Congolese Franc in AO
Searching: Congolese Franc in AG
Searching: Congolese Franc in AR
Searching: Congolese Franc in AM
Searching: Congolese Franc in AU
Searching: Congolese Franc in AT
Searching: Congolese Franc in AZ
Searching: Congolese Franc in BS
Searching: Congolese Franc in BH
Searching: Congolese Franc in BD
Searching: Congolese Franc in BB
Searching: Congolese Franc in BY
Searching: Congolese Franc in BE
Searching: Congolese Franc in BZ
Searching: Congolese Franc in BJ
Searching: Congolese Franc in BT
Searching: Congolese Franc in BO
Searching: Congolese Franc in BA
Searching: Congolese Franc in BW
Searching: Congolese Franc in BR
Searching: Congolese Franc in BN
Searching: Congolese Franc in BG
Searching: Congolese Franc in BF
Searching: Congolese Franc in BI
Searching: Congolese Franc in CV
Searching: Congolese Franc in KH
Searching:

RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22New+Zealand+Dollar%22%2C+%22time%22%3A+%22all%22%2C+%22geo%22%3A+%22MR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))

In [18]:
results

,date,trends_index,keyword,region
0,2004-01-01,68,Congolese Franc,CD
1,2004-02-01,0,Congolese Franc,CD
2,2004-03-01,0,Congolese Franc,CD
3,2004-04-01,0,Congolese Franc,CD
4,2004-05-01,0,Congolese Franc,CD
...,...,...,...,...
228,2023-01-01,5,New Zealand Dollar,MY
229,2023-02-01,6,New Zealand Dollar,MY
230,2023-03-01,5,New Zealand Dollar,MY
231,2023-04-01,4,New Zealand Dollar,MY


In [19]:
results.to_csv("currency_partial results/currency17.csv")

Loop modifying IP address:

In [13]:
import random
from pytrends.request import TrendReq
import time
import ipaddress


# Set up the Google Trends API object

# Initialize an empty DataFrame to store the results
results = pd.DataFrame()

ip_address = ipaddress.IPv4Address('192.168.1.1')  # create an IPv4Address object with initial IP address


# Loop through regions and keywords
for keyword in keywords_list:
    for i, country in enumerate(countries):
        
        if i % 25 == 0:  # modify the IP address every 25th iteration
            new_last_octet = ip_address.packed[-1] + random.randint(1, 255)
            new_ip_address = ipaddress.IPv4Address(ip_address.packed[:-1] + bytes([new_last_octet]))
            ip_address = new_ip_address
            print("IP address:" + str(ip_address))
        else:  
            new_ip_address = ip_address
        
        

        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(1)
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])



IP address:192.168.1.119
Searching: Barbados Dollar in AF
IP address:192.168.1.119
Searching: Barbados Dollar in AL
IP address:192.168.1.119
Searching: Barbados Dollar in DZ
IP address:192.168.1.119
Searching: Barbados Dollar in AD
IP address:192.168.1.119
Searching: Barbados Dollar in AO
IP address:192.168.1.119
Searching: Barbados Dollar in AG
IP address:192.168.1.119
Searching: Barbados Dollar in AR
IP address:192.168.1.119
Searching: Barbados Dollar in AM
IP address:192.168.1.119
Searching: Barbados Dollar in AU


KeyboardInterrupt: 

In [14]:
import socket

hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
print(ip_address)

192.168.1.35


In [18]:
from requests import Session
from requests_ip_rotator import RotatingIP

from pytrends.request import TrendReq



ImportError: cannot import name 'RotatingIP' from 'requests_ip_rotator' (c:\Users\danid\anaconda3\envs\master_thesis\Lib\site-packages\requests_ip_rotator\__init__.py)

In [ ]:
# List of IP addresses to rotate through
ip_list = ['1.2.3.4', '5.6.7.8', '9.10.11.12']

# Create a session with a rotating IP adapter
s = Session()
s.mount('http://', RotatingIP(ip_list))

# Create a TrendReq object with the rotating IP session
pytrends = TrendReq(session=s)

# Use the pytrends object as normal to get Google Trends data
pytrends.build_payload(kw_list=['Python'])
trend_data = pytrends.interest_over_time()
print(trend_data)
